#Урок 3. Логистическая регрессия. Log Loss

In [1]:
import numpy as np

1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).  


In [54]:
def calc_logloss(y, p):
    p = np.clip(p, 0.01, 0.99)
    err = np.mean(- y * np.log(p) - (1.0 - y) * np.log(1.0 - p))
    return err

In [58]:
# Пример применения
y_true = 0
y_pred = 0
calc_logloss(y_true, y_pred)

0.01005033585350145

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss не превышал значение `0.3` `(0.1).` Как изменились веса?


In [19]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def standardization(x: np.ndarray) -> np.ndarray:
    s = (x - x.mean()) / x.std()
    return s

X_st = X.copy()
X_st[:, 2] = standardization(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [43]:
# логистическая регрессия
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [59]:
# Изначальное число итераций и скорость обучения:
w = eval_LR_model(X_st, y, 1000, 1e-1)

100 [-0.29848458 -0.72686866  1.06314016  1.39690737] 0.5235767485993659
200 [-0.64245884 -0.81575707  1.11680356  1.72702881] 0.4995552177931558
300 [-0.95659618 -0.87689661  1.10232809  2.00636498] 0.4814150662731218
400 [-1.25432451 -0.91736333  1.05455726  2.24457861] 0.4664467561141666
500 [-1.5375957  -0.94440497  0.98735478  2.45373702] 0.4535011239613408
600 [-1.80755433 -0.96270374  0.90904148  2.64183838] 0.44201377632900957
700 [-2.06512906 -0.9753011   0.82492184  2.81420765] 0.4316738793739683
800 [-2.31113497 -0.98420861  0.73842971  2.97447927] 0.42228858977623335
900 [-2.54631605 -0.99077064  0.65178699  3.12518707] 0.4137239929669819
1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734


In [61]:
# Число итераций увеличено втрое, значение ошибки уменьшается только до 0,33:
w = eval_LR_model(X_st, y, 3000, 1e-1)

300 [-0.95659618 -0.87689661  1.10232809  2.00636498] 0.4814150662731218
600 [-1.80755433 -0.96270374  0.90904148  2.64183838] 0.44201377632900957
900 [-2.54631605 -0.99077064  0.65178699  3.12518707] 0.4137239929669819
1200 [-3.193644   -1.00399774  0.40259598  3.53557456] 0.3920245709386051
1500 [-3.76622998 -1.01499348  0.17863749  3.90176516] 0.3749222940537399
1800 [-4.27794785 -1.0270723  -0.01920407  4.23624356] 0.36113578827337056
2100 [-4.74008164 -1.0407042  -0.19438587  4.54566439] 0.3497829571572578
2400 [-5.16163889 -1.05560731 -0.3509117   4.83434101] 0.3402479132898681
2700 [-5.54974842 -1.07139367 -0.49227314  5.10546336] 0.332095456168512
3000 [-5.91004767 -1.08773886 -0.62127728  5.36155604] 0.3250134761492536


In [62]:
# Число итераций увеличено в 10 раз, значение ошибки достигает 0,3 к 4000-й итерации:
w = eval_LR_model(X_st, y, 10000, 1e-1)

1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734
2000 [-4.59094777 -1.0359998  -0.13825631  4.44501211] 0.35333924193451943
3000 [-5.91004767 -1.08773886 -0.62127728  5.36155604] 0.3250134761492536
4000 [-6.96133333 -1.14371437 -0.98694403  6.13057114] 0.3066706532490403
5000 [-7.85498033 -1.19935714 -1.28748713  6.80564285] 0.293563042595275
6000 [-8.64731257 -1.25347799 -1.54786532  7.41682354] 0.2830505243174329
7000 [-9.36966717 -1.30586395 -1.78130777  7.98193089] 0.27438119140864203
8000 [-10.04075209  -1.35657036  -1.99546097   8.51208184] 0.2669168952629414
9000 [-10.67239108  -1.40571631  -2.1950503    9.01453862] 0.2601957877911167
10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.25414729511686257


In [64]:
# Число итераций увеличено в 20 раз, значение ошибки так и не достигает 0,1:
w = eval_LR_model(X_st, y, 20000, 1e-1)

2000 [-4.59094777 -1.0359998  -0.13825631  4.44501211] 0.35333924193451943
4000 [-6.96133333 -1.14371437 -0.98694403  6.13057114] 0.3066706532490403
6000 [-8.64731257 -1.25347799 -1.54786532  7.41682354] 0.2830505243174329
8000 [-10.04075209  -1.35657036  -1.99546097   8.51208184] 0.2669168952629414
10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.25414729511686257
12000 [-12.39769211  -1.54494868  -2.73269053  10.39838241] 0.2435837204422556
14000 [-13.44424765  -1.63184527  -3.05468703  11.24281649] 0.23426726382407703
16000 [-14.4278092   -1.71463311  -3.35508641  12.03835138] 0.22594342826095087
18000 [-15.35833092  -1.7937128   -3.63759762  12.79206658] 0.21844107429040505
20000 [-16.2427091   -1.86940882  -3.90475531  13.50901763] 0.21163523164770007


С увеличением числа итераций ошибка уменьшается. Также в общем уменьшаются веса, кроме веса: "квалификация репититора", этот вес наоборот увеличивается. После 10000-й итерации скорость уменьшения ошибки резко падает.

In [67]:
# Скорость обучения уменьшена вдвое при исходном числе итераций,
# значение ошибки, практически, не уменьшается:
w = eval_LR_model(X_st, y, 1000, 1e-2)

100 [ 0.25769171 -0.68297243  0.68841418  1.2405903 ] 0.5908596023453088
200 [ 0.14805533 -0.69317972  0.77965488  1.21287192] 0.5687278067099054
300 [ 0.06257746 -0.68605867  0.85181735  1.21283095] 0.5560473747533587
400 [-0.00887935 -0.68415387  0.90713736  1.22443854] 0.5476800278385013
500 [-0.0701621  -0.68661758  0.94984062  1.24404686] 0.5416765996151559
600 [-0.1239972  -0.69202069  0.98318257  1.26924759] 0.5369841864977791
700 [-0.17239277 -0.69933957  1.00953865  1.29824779] 0.5330427663756572
800 [-0.21681092 -0.7078635   1.03062761  1.32973537] 0.5295551503580799
900 [-0.25830834 -0.71709934  1.04769231  1.36276535] 0.5263621224044184
1000 [-0.29764618 -0.72670545  1.061634    1.39666497] 0.5233765331724747


Веса в общем увеличиваются, кроме веса: "квалификация репититора", этот вес наоборот уменьшается.

In [68]:
# Скорость обучения увеличена на один порядок при исходном числе итераций,
# значение ошибки уменьшается с гораздо большей скоростью,
# искомое значение ошибки 0,1 достигается уже между 200-й и 300-й итерациями:
w = eval_LR_model(X_st, y, 1000, 1e1)

100 [-36.15930941 -20.32927872   4.23267152  38.50456495] 1.9184987864955012
200 [-50.13974386  -5.25681348 -13.47526141  43.09961333] 0.1694839308870087
300 [-57.59828441  -6.08259973 -15.38056431  47.80839137] 0.08404215077341365
400 [-58.75172063  -5.5584856  -16.18089442  47.63223568] 0.08154793949473194
500 [-61.33919466  -7.84836273 -14.62815829  53.55189664] 0.09998404341520625
600 [-63.19792482  -6.50370154 -17.08144314  52.09182185] 0.07764072469065969
700 [-64.2140392   -6.01423707 -17.75007538  51.94137878] 0.07587894673802852
800 [-64.92929338  -5.92856046 -18.07731712  52.23821266] 0.07517248972106222
900 [-65.55757621  -5.9453491  -18.28789585  52.66781283] 0.07460699328624897
1000 [-66.16312453  -5.98180042 -18.47328021  53.11963154] 0.07406125457273503


С увеличением скоростии обучения веса опять в общем уменьшаются, а вес: "квалификация репититора" увеличивается.

3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`.


In [73]:
def calc_pred_proba (X, w):
  pred = sigmoid(np.dot(X, w))
  return pred.round(2)

calc_pred_proba(X_st, w)

array([0.28, 0.  , 1.  , 0.01, 0.98, 0.  , 1.  , 0.  , 0.73, 1.  ])

4. Создайте функцию `calc_pred`, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, а также порог вероятности.

In [74]:
def calc_pred (X, w):
  pred = sigmoid(np.dot(X, w))
  pred.round(2)
  res = np.where(pred > 0.5, 1, 0)
  return res

calc_pred(X_st, w)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])